In [5]:
import numpy as np

In [6]:
class BinaryNeuralNetwork:
    def __init__(self, input_size: int, hidden_layers: int, nodes_in_each_layer: int):
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.nodes_in_hidden_layer = nodes_in_each_layer

        self.weights = [np.random.randn(input_size, nodes_in_each_layer) / np.sqrt(input_size)]
        self.bias = [np.random.randn(nodes_in_each_layer, 1)]

        for _ in range(hidden_layers - 1):
            self.weights.append(np.random.randn(nodes_in_each_layer, nodes_in_each_layer) / np.sqrt(input_size))
            self.bias.append(np.random.randn(nodes_in_each_layer, 1))
        
        self.weights.append(np.random.randn(nodes_in_each_layer, 1) / np.sqrt(input_size))
        self.bias.append(np.random.randn(1, 1))

    
    def forward_propagate(self, input_xs: np.ndarray):
        self.num_examples = input_xs.shape[1]
        self.activations = [input_xs]
        self.Z = []
        for i in range(0, self.hidden_layers + 1):
            Z: np.ndarray = self.weights[i].T.dot(self.activations[i]) + self.bias[i]
            if i != self.hidden_layers:
                assert Z.shape == (self.nodes_in_hidden_layer, self.num_examples)
            else:
                assert Z.shape == (1, self.num_examples)

            activation = leaky_relu if i != self.hidden_layers else sigmoid
            self.activations.append(activation(Z))
            self.Z.append(Z)

        self.output = self.activations[-1]
        assert self.output.shape == (1, self.num_examples)
        # print(self.output)

    
    def backpropagate(self, inp_ys: np.ndarray, learning_rate: float):
        (dL_dOutputW, dL_dOutputB, dL_dA_Lminus1) = self.output_loss(inp_ys)
        (dWeights, dBiases) = self.hidden_loss(dL_dA_Lminus1)
        # print(f"dWeights = {dWeights}\n\ndBiases = {dBiases}")

        self.weights[-1] -= learning_rate * dL_dOutputW
        self.bias[-1] -= learning_rate * dL_dOutputB


        for i in range(self.hidden_layers):
            # print(i)
            self.weights[i] -= learning_rate * dWeights[i]
            self.bias[i] -= learning_rate * dBiases[i]
            
    

    def output_loss(self, inp_ys: np.ndarray) -> (np.ndarray, np.ndarray, np.ndarray):
        # ------------------ For Output Layer, ---------------------------------------

        # print(f"inp_ys = {inp_ys.shape} && self.output == {self.output.shape}")
        assert inp_ys.shape == self.output.shape

        dL_dOutputA = -(inp_ys - self.output)
        assert dL_dOutputA.shape == (1, self.num_examples)
        dA_dOutputZ = sigmoid_derivative(self.activations[-1])
        assert dA_dOutputZ.shape == (1, self.num_examples)
        
        dL_dOutputZ = dL_dOutputA * dA_dOutputZ
        assert dL_dOutputZ.shape == (1, self.num_examples)

        dOutputZ_dW = self.activations[-2]
        assert dOutputZ_dW.shape == (self.nodes_in_hidden_layer, self.num_examples)

        # dL_dOutputZ should be broadcasted up for each of the weights
        dL_dOutputW = dL_dOutputZ * dOutputZ_dW
        assert dL_dOutputW.shape == (self.nodes_in_hidden_layer, self.num_examples)
        dL_dOutputW = np.sum(dL_dOutputW, axis=1, keepdims=True) / self.num_examples
        assert dL_dOutputW.shape == (self.nodes_in_hidden_layer, 1)

        # print(f"dL_dOutputW = {dL_dOutputW}")

        dL_dOutputB = dL_dOutputZ # (dOutuptZ_dB == 1)
        dL_dOutputB = np.sum(dL_dOutputB, axis=1, keepdims=True) / self.num_examples
        assert dL_dOutputB.shape == (1, 1)
        # print(f"dL_dOutputB = {dL_dOutputB}")

        # -------------------- Done For Output Layer ---------------------------------------
        # Now those are corrections made for this layer, but to propagate error backwards,
        # we need to calculate error for the previous nodes as well.

        dL_dA_l_1 = dL_dOutputZ * self.weights[-1] # dOutputZ_dA_l_1
        assert dL_dA_l_1.shape == (self.nodes_in_hidden_layer, self.num_examples)

        return (dL_dOutputW, dL_dOutputB, dL_dA_l_1)
    

    def hidden_loss(self, dLoss_dA_Lminus1: np.ndarray) -> (list[np.ndarray], list[np.ndarray]):
        # dLoss_dA_curr_layer is done for all self.num_examples.
        dLoss_dA_curr_layer = dLoss_dA_Lminus1
        # print(f"dLoss_dA_Lminus1 = {dLoss_dA_curr_layer}")
        assert dLoss_dA_curr_layer.shape == (self.nodes_in_hidden_layer, self.num_examples)
        
        dLoss_dWs = []
        dLoss_dBs = []

        for i in range(self.hidden_layers - 1, -1, -1):
            dA_dZ = leaky_relu_derivative(self.Z[i])
            assert dA_dZ.shape == (self.nodes_in_hidden_layer, self.num_examples)
            
            dLoss_dZ = dLoss_dA_curr_layer * dA_dZ
            assert dLoss_dZ.shape == (self.nodes_in_hidden_layer, self.num_examples)

            dZ_dW = self.activations[i]
            # Really, it should be a 3D array, but it would actually just be a broadcast of each column, putting the examples in the 3rd dimension.
            if i != 0:
                assert dZ_dW.shape == (self.nodes_in_hidden_layer, self.num_examples)
            else:
                assert dZ_dW.shape == (self.input_size, self.num_examples)

            # What's happening: dLoss_dZ's outer product with dZ_dW and then averaged over all training examples.
            # dZ_dW is actually supposed to be a 2D matrix for one example.
            # dLoss_dZ is supposed to be a 1D vector for one example.
            # dLoss_dZ = [dLoss_dZ1, dLoss_dZ2, ....]
            # dZ_dW    =[[dZ_dW00,   dZ_dW01,   dZ_dW02, ..]
            #            [dZ_dW10,   dZ_dW11,   dZ_dW12, ..]
            #               ...
            #           ]
            # But if you think about it, it's actually just:
            # dZ_dW    =[[A(L-2, 0), A(L-2, 0), A(L-2, 0), ..]
            #            [A(L-2, 1), A(L-2, 1), A(L-2, 1), ..]
            #            ...
            #           ]
            # So really, for one example, it's just a 1D vector broadcasted into 2D.                    VVVV 1D part             , 1D part for all examples
            # Hence our dZ_dW is technically having all information needed since its dimensions are (self.num_nodes_in_each_layer, self.num_examples).
            # So, what we really want dLoss_dW to be is outer product of dLoss_dZ (1D form) and dZ_dW (1D form).
            # and then since we have training examples, we average it over all training examples.
            # Finally, the resulting weight differential should be arranged as (prev_layer X current_layer),
            # and the resulting weight differential out of dL/dZ @ dZ/dW is actually (current_layer X prev_layer), 
            # so instead we do dZ/dW @ dL/dZ.
            # This is what the simple dot product expression is doing. Magic to me how it all worked out almost coincidentally so well.
            dLoss_dW = (dZ_dW.dot(dLoss_dZ.T)) / self.num_examples
            if i != 0:
                assert dLoss_dW.shape == (self.nodes_in_hidden_layer, self.nodes_in_hidden_layer)
            else:
                assert dLoss_dW.shape == (self.input_size, self.nodes_in_hidden_layer)
            
            dLoss_dWs.append(dLoss_dW)

            dLoss_dB = np.sum(dLoss_dZ, axis=1, keepdims=True) / self.num_examples
            assert dLoss_dB.shape == (self.nodes_in_hidden_layer, 1)

            dLoss_dBs.append(dLoss_dB)

            dLoss_dA_curr_layer = self.weights[i].dot(dLoss_dZ)
            
            # print(f"dLoss_dA_curr_layer = {dLoss_dA_curr_layer}")
            
            if i != 0:
                assert dLoss_dA_curr_layer.shape == (self.nodes_in_hidden_layer, self.num_examples)
            else:
                assert dLoss_dA_curr_layer.shape == (self.input_size, self.num_examples)

        return (list(reversed(dLoss_dWs)), list(reversed(dLoss_dBs)))
            

def sigmoid(x):
    # Check if any element in the matrix is > 100_000
    if np.any(x > 100_000):
        print(x)
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(g_x):
    return g_x * (1 - g_x)

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x < 0, 0, 1)

def leaky_relu(x):
    return np.maximum(0.01*x, x)

def leaky_relu_derivative(x: np.ndarray):
    return np.where(x < 0, 0.01, 1)

In [11]:
# Create a neural network to determine if first number is a factor of second or not.
nn = BinaryNeuralNetwork(2, 10, 5)
# Generate training data.
examples = 100_000
xs = np.random.randint(0., 10., (2, examples))
ys = np.array([float(int(x[1] % x[0] == 0)) for x in xs.T]).reshape(1, examples)
# Split into training and testing data as a fraction of examples
train_xs = xs[:, :int(0.8 * examples)]
train_ys = ys[:, :int(0.8 * examples)]
print(train_xs, train_ys)
test_xs = xs[:, int(0.8 * examples):]
test_ys = ys[:, int(0.8 * examples):]
# Train the neural network with 500 examples at a time
for _ in range(150):
    for i in range(0, train_xs.shape[1], 1000):
        nn.forward_propagate(train_xs[:, i:i+1000])
        nn.backpropagate(train_ys[:, i:i+1000], 0.01)


    nn.forward_propagate(test_xs)
    print(f"Output = {nn.output}")
    print(f"Expected = {test_ys}")
    print(f"Accuracy = {np.sum(np.round(nn.output) == test_ys) / test_ys.shape[1]}")
    print(f"Number of one classes = {np.sum(test_ys)}, total examples = {test_ys.shape[1]}")
    # Test the neural network


print(nn.weights)
print(nn.bias)

/tmp/ipykernel_43483/290922660.py:6: RuntimeWarning: divide by zero encountered in long_scalars
  ys = np.array([float(int(x[1] % x[0] == 0)) for x in xs.T]).reshape(1, examples)


[[0 4 0 ... 7 1 6]
 [7 0 6 ... 5 0 9]] [[1. 1. 1. ... 0. 1. 0.]]
Output = [[1.         0.99999987 1.         ... 0.99987295 0.99966877 0.99987295]]
Expected = [[0. 0. 1. ... 1. 1. 1.]]
Accuracy = 0.4237
Number of one classes = 8474.0, total examples = 20000

[array([[ 0.90930575, -0.1311623 , -0.20493618, -0.85045517, -0.97454823],
       [ 1.29981292, -0.32923034, -0.40592994,  0.07651804,  1.36811296]]), array([[-0.67342477,  0.34660589,  1.21908679,  1.1399262 , -1.64668184],
       [ 1.18298334,  0.05499343, -0.61664143, -0.01833464, -0.71349218],
       [ 0.72363393, -0.12664726,  0.02639522,  1.98884081, -0.99404407],
       [ 0.04003145,  0.47740134,  1.31017365,  0.1279123 , -1.33018454],
       [-1.1148108 ,  1.22285917, -1.40125041, -1.30061228,  0.79779575]]), array([[-0.25951014, -0.3894772 ,  0.57745053, -0.47728579, -0.21301921],
       [ 0.59761615,  0.61402464,  0.10752936, -0.78696983, -0.28819571],
       [-0.16750222,  1.02889863,  0.01868759, -0.1891239 , -1.5777929